In [ ]:
import torch
import torchvision
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn as nn
from torch import optim
from torch.autograd import Variable

In [ ]:
learning_rate = 1e-3
batch_size = 128
epoches = 50

if torch.cuda.is_available():
    print(torch.cuda.device_count())
    print("CUDA inside")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
simple_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(), # The order should be right
    transforms.Normalize([0.485, 0.456, 0.406],
                        [0.229, 0.224, 0.225])
                     ])
train = ImageFolder('/home/kost14/DL_with_Pytorch/Chapter3/dogs-vs-cats/train', simple_transform)
valid = ImageFolder('/home/kost14/DL_with_Pytorch/Chapter3/dogs-vs-cats/valid', simple_transform)
# test = ImageFolder('/home/kost14/DL_with_Pytorch/Chapter3/dogs-vs-cats/test1', simple_transform)

In [ ]:
trainLoader = DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=4)
validLoader = DataLoader(valid, batch_size=batch_size, shuffle=False, num_workers=4)
# testLoader = DataLoader(test, batch_size=batch_size, shuffle=False, num_workers=4)

In [ ]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        
        self.conv = nn.Sequential(
        nn.Conv2d(3, 96, 11, stride=4), # 1
        nn.ReLU(inplace=True),
        nn.MaxPool2d(3, 2),
        nn.Conv2d(96, 256, 5, padding=2), # 2
        nn.ReLU(inplace=True),
        nn.MaxPool2d(3, 2),
        nn.Conv2d(256, 384, 3, padding=1), # 3
        nn.ReLU(inplace=True),
        nn.Conv2d(384, 384, 3, padding=1), # 4
        nn.ReLU(inplace=True),
        nn.Conv2d(384, 256, 3, padding=1), # 5
        nn.ReLU(inplace=True),
        nn.MaxPool2d(3, 2)
        )
        
        
        self.fc = nn.Sequential(
        nn.Linear(256*5*5, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5),
        nn.Linear(4096, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5),
        nn.Linear(4096, 2)
        )

    def forward(self, x):
        x = self.conv(x)
#         print(x.shape)
        x = x.view(x.size(0), 256*5*5)
        x = self.fc(x)
        return x

In [ ]:
alexnet = AlexNet()
if torch.cuda.is_available():
    alexnet.cuda()

In [ ]:
criterian = nn.CrossEntropyLoss()
optimizer = optim.SGD(alexnet.parameters(), lr=learning_rate, momentum=0.9)

In [ ]:
for epoch in range(epoches):
    sum_loss_t = 0.0
    sum_acc_t = 0.0
    sum_loss_v = 0.0
    sum_acc_v = 0.0
    
    for (img, label) in trainLoader:
        img = img.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()
        output = alexnet(img)
        loss = criterian(output, label)
        # backward
        loss.backward()
        optimizer.step()
        
        sum_loss_t += loss.item()
        _, predict = torch.max(output, 1)
        correct_num = (predict==label).sum()
        sum_acc_t += correct_num.item()
    
#     for (img, label) in validLoader:
#         img = img.to(device)
#         label = label.to(device)
        
#         optimizer.zero_grad()
#         output = alexnet(img)
#         loss = criterian(output, label)
#         # backward
#         loss.backward()
#         optimizer.step()
        
#         sum_loss_v += loss.item()
#         _, predict = torch.max(output, 1)
#         correct_num = (predict==label).sum()
#         sum_acc_v += correct_num.item()
    
    sum_loss_t /= len(train)
    sum_acc_t /= len(train)
#     sum_loss_v /= len(valid)
#     sum_acc_v /= len(valid)
    print("[%d/%d] Loss: %.5f, Acc: %.2f" %(epoch+1, epoches, sum_loss_t, 100*sum_acc_t))
#     print("[%d/%d] Loss: %.5f, Acc: %.2f" %(epoch+1, epoches, sum_loss_v, 100*sum_acc_v))    